In [1]:
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.fft import fft, fftfreq

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Список файлов
file_paths = ["data/EEG Data/eeg_record1.mat", "data/EEG Data/eeg_record2.mat", "data/EEG Data/eeg_record3.mat", "data/EEG Data/eeg_record4.mat", "data/EEG Data/eeg_record5.mat", "data/EEG Data/eeg_record6.mat", "data/EEG Data/eeg_record7.mat", 
              "data/EEG Data/eeg_record8.mat", "data/EEG Data/eeg_record9.mat", "data/EEG Data/eeg_record10.mat", "data/EEG Data/eeg_record11.mat", "data/EEG Data/eeg_record12.mat", "data/EEG Data/eeg_record13.mat", "data/EEG Data/eeg_record14.mat", 
              "data/EEG Data/eeg_record15.mat", "data/EEG Data/eeg_record16.mat", "data/EEG Data/eeg_record17.mat", "data/EEG Data/eeg_record18.mat", "data/EEG Data/eeg_record19.mat", "data/EEG Data/eeg_record20.mat", "data/EEG Data/eeg_record21.mat" ,
              "data/EEG Data/eeg_record22.mat", "data/EEG Data/eeg_record23.mat","data/EEG Data/eeg_record24.mat", "data/EEG Data/eeg_record25.mat", "data/EEG Data/eeg_record26.mat", "data/EEG Data/eeg_record27.mat", "data/EEG Data/eeg_record28.mat", 
              "data/EEG Data/eeg_record29.mat", "data/EEG Data/eeg_record30.mat", "data/EEG Data/eeg_record31.mat", "data/EEG Data/eeg_record32.mat", "data/EEG Data/eeg_record33.mat", "data/EEG Data/eeg_record34.mat"]  # Укажите пути к вашим 34 файлам


In [2]:
fs = 128 
n_subjects = 5
mkpt1 = int(fs*10*60)
mkpt2 = int(fs*20*60)

In [3]:
subject_map = {}
for s in range(1, n_subjects+1):
    a =  int(7*(s-1)) + 3
    if s!=5:
        b = a + 5
    else:
        b = a + 4
    subject_map[s] = [i for i in range(a, b)]
print(subject_map)

{1: [3, 4, 5, 6, 7], 2: [10, 11, 12, 13, 14], 3: [17, 18, 19, 20, 21], 4: [24, 25, 26, 27, 28], 5: [31, 32, 33, 34]}


In [4]:
channels = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']
useful_channels = ['F7','F3','P7','O1','O2','P8','AF4']
use_channel_inds = []
for c in useful_channels:
    if c in channels:
        use_channel_inds.append(channels.index(c))

In [5]:
use_channel_inds

[1, 2, 5, 6, 7, 8, 13]

In [10]:
inp_dir = 'data/EEG Data/' 

In [11]:
from scipy.io import loadmat
import pickle

print(mkpt1)
print(mkpt2)
mkpt3 = 214540
interval = mkpt3 - mkpt2
for s in range(1, n_subjects+1):
    data = {}
    data['channels'] = useful_channels
    data['fs'] = fs
    for i, t in enumerate(subject_map[s]):
        trial = {}
        trial_data = loadmat(inp_dir + f'eeg_record{t}.mat')
        eeg = trial_data['o']['data'][0][0][:, 3:17]
        eeg = eeg[:, use_channel_inds]
        
        trial['focussed'] = eeg[:interval]
        trial['unfocussed'] = eeg[mkpt1:mkpt1+interval]
        trial['drowsed'] = eeg[mkpt2:mkpt2+interval]
        data[f'trial_{i+1}'] = trial
    with open(f'data/sub/subject_{s}.pkl', 'wb') as f: 
        pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
        

76800
153600


In [ ]:
with open('data/sub/subject_1.pkl', 'rb') as f: 
    data1 = pickle.load(f)
with open('data/sub/subject_2.pkl', 'rb') as f: 
    data2 = pickle.load(f)
with open('data/sub/subject_3.pkl', 'rb') as f: 
    data3 = pickle.load(f)
with open('data/sub/subject_4.pkl', 'rb') as f: 
    data4 = pickle.load(f)
with open('data/sub/subject_5.pkl', 'rb') as f: 
    data5 = pickle.load(f)

In [3]:
FS = 128  # Частота дискретизации
participant_data = {}  # Словарь для хранения данных по участникам

for file_path in file_paths:
    data = scipy.io.loadmat(file_path)
    eeg_data = data['o']['data'][0, 0]  # Извлечение данных

    # Извлечение ID участника
    participant_id = data['o']['id'][0, 0][0]  # Получение строки из массива
    if participant_id not in participant_data:
        participant_data[participant_id] = []

    # Определение каналов и полезных каналов
    channels = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']
    useful_channels = ['F7', 'F3', 'P7', 'O1', 'O2', 'P8', 'AF4']
    use_channel_inds = [channels.index(c) for c in useful_channels]

    # Извлечение данных для полезных каналов
    filtered_data = eeg_data[:, use_channel_inds]

    # Убираем каналы с околонулевыми значениями, считаем их шумами
    # Удаление каналов F3 и F7
    filtered_data = np.delete(filtered_data, [0, 1], axis=1)  # Удаляем F3 и F7 (индексы 0 и 1)

    # Разметка данных по состояниям
    states = {
        'focused': filtered_data[:FS * 10 * 60, :],  # 0-10 минут
        'unfocused': filtered_data[FS * 10 * 60:FS * 20 * 60, :],  # 10-20 минут
        'drowsy': filtered_data[FS * 20 * 60:, :]  # 20 минут и далее
    }

    # Добавление данных участника
    participant_data[participant_id].append(states)

# Обработка для группировки данных по участникам и экспериментам
grouped_data = {}
for participant_id, experiments in participant_data.items():
    if participant_id not in grouped_data:
        grouped_data[participant_id] = {'focused': [], 'unfocused': [], 'drowsy': []}
    
    # Сбор данных по состояниям
    for states in experiments:
        grouped_data[participant_id]['focused'].append(states['focused'])
        grouped_data[participant_id]['unfocused'].append(states['unfocused'])
        grouped_data[participant_id]['drowsy'].append(states['drowsy'])

# Пример: вывести сгруппированные данные для каждого участника
for participant_id, states in grouped_data.items():
    print(f"\nУчастник: {participant_id}")
    for state_name, state_data in states.items():
        # Усреднение данных по состояниям
        mean_state_data = np.mean(np.array(state_data), axis=0)
        print(f"\nСостояние: {state_name.capitalize()}")
        print(f"Усреднённые данные (по каналам):\n{mean_state_data}")


Участник: 201410092013.D091BB44

Состояние: Focused
Усреднённые данные (по каналам):
[[5390.76923077 3833.84615385 4019.48717949 4656.41025641 4125.64102564]
 [5389.23076923 3830.25641026 4020.         4658.97435897 4136.41025641]
 [5385.64102564 3829.74358974 4017.94871795 4655.8974359  4152.82051282]
 ...
 [4825.12820513 3826.15384615 4016.41025641 4125.12820513 4138.97435897]
 [4814.87179487 3829.23076923 4018.46153846 4119.48717949 4142.05128205]
 [4814.35897436 3828.71794872 4019.48717949 4119.48717949 4139.48717949]]

Состояние: Unfocused
Усреднённые данные (по каналам):
[[4815.8974359  3828.71794872 4017.94871795 4122.56410256 4140.        ]
 [4811.28205128 3829.23076923 4014.87179487 4118.97435897 4142.56410256]
 [4807.69230769 3826.15384615 4014.87179487 4117.43589744 4136.41025641]
 ...
 [5054.35897436 3827.17948718 4022.05128205 4301.02564103 4134.35897436]
 [5056.41025641 3828.20512821 4022.56410256 4300.         4135.8974359 ]
 [5051.28205128 3829.74358974 4023.07692308 4

# Дальше будет визуализация всех сигналов сгруппированными по состоянию человека, для каждого участника.
(Визуализация очень подробная, поэтому если интересно увидеть каждый из графиков, то вставьте код в ячейку)

```py
import matplotlib.pyplot as plt

experiment_days = {
    '20141009': ['201410092013.D091BB44', '201410092014.D091BB44'],
    '20141010': ['201410102013.D091BB44', '201410102014.D091BB44'],
    '20141011': ['201410112013.D091BB44', '201410112014.D091BB44'],
    '20141012': ['201410122013.D091BB44', '201410122014.D091BB44'],
    '20141013': ['201410132013.D091BB44', '201410132014.D091BB44'],
}

# Визуализация сигналов по дням
for participant_id, states in grouped_data.items():
    plt.figure(figsize=(12, 6))
    plt.suptitle(f'Сигналы для участника: {participant_id}', fontsize=16)
    
    for state_name, state_data in states.items():
        plt.subplot(3, 1, list(states.keys()).index(state_name) + 1)
        # Усреднение данных по состояниям
        mean_state_data = np.mean(np.array(state_data), axis=0)
        
        for channel_index in range(mean_state_data.shape[1]):
            plt.plot(mean_state_data[:, channel_index], label=f'Канал {channel_index + 1}')
        
        plt.title(state_name.capitalize())
        plt.xlabel('Время (сэмплы)')
        plt.ylabel('Амплитуда')
        plt.legend()
        plt.grid(True)

    plt.tight_layout(rect=[0, 0, 1, 0.96])  # Подгонка под заголовок
    plt.show()
```

Небольшие выводы по большой визуализации: 
* можно заметить, что 2 канал всегда лежит в районе 0, из-за чего можно сделать вывод, что во время эксперимента этот канал был неисправен.
* выше всех по амплитуде всегда 3 канал(в районе 5000 для focused, 4700-5000 для unsfocused, и около 4500-4700 для drowsy), ниже него обычно 6 канал.
* для большинства каналов всех 3-х состояний амплитуда обычно 4000.

In [5]:
week_data = {}  # Словарь для хранения данных за неделю для каждого участника

for participant_id, states in grouped_data.items():
    week_data[participant_id] = {}

    for state_name, state_data in states.items():
        # Сконкатенируем данные по всем экспериментам для данного состояния
        all_data = np.concatenate(state_data, axis=0)
        week_data[participant_id][state_name] = all_data

```py
import seaborn as sns

week_data = {}  # Словарь для хранения данных за неделю для каждого участника

for participant_id, states in grouped_data.items():
    week_data[participant_id] = {}

    for state_name, state_data in states.items():
        # Сконкатенируем данные по всем экспериментам для данного состояния
        all_data = np.concatenate(state_data, axis=0)
        week_data[participant_id][state_name] = all_data

# Визуализация корреляции сигналов
for participant_id, states in week_data.items():
    plt.figure(figsize=(12, 8))
    plt.suptitle(f'Корреляция сигналов для участника: {participant_id}', fontsize=16)

    for state_name, state_data in states.items():
        # Вычисление корреляционной матрицы
        corr_matrix = np.corrcoef(state_data, rowvar=False)

        plt.subplot(3, 1, list(states.keys()).index(state_name) + 1)
        sns.heatmap(corr_matrix, cmap='coolwarm', annot=False, fmt=".2f",
                    xticklabels=useful_channels, yticklabels=useful_channels, cbar_kws={'label': 'Correlation Coefficient'})
        plt.title(f'Корреляция для состояния: {state_name.capitalize()}')
        plt.xlabel('Каналы')
        plt.ylabel('Каналы')

    plt.tight_layout(rect=[0, 0, 1, 0.96])  # Подгонка под заголовок
    plt.show()
```

Небольшие выводы по визуализации корреляции: 
* в почти в каждом графике максимальная корреляция достигается между P7 и Р8, также она скачет от 0.5 до 0.97
* в остальном данные почти не коррелируют

## Очистка данных 
А именно: От сломанных датчиков (значение не меняется на протяжении всего эксперимента +- отклонение меньше 3%)

In [6]:
import numpy as np

# порог для стандартного отклонения
threshold = 0.03  # Порог, который можно настроить

# Инициализация переменной для хранения данных после удаления неинформативного канала
filtered_participant_data = {}

for participant_id, states in participant_data.items():
    filtered_states = {}
    
    for state in states:  # Предполагаем, что states — это список
        # Пример: state — это словарь с данными для каждого состояния
        # Убедитесь, что каждый элемент состояния является словарем
        for state_name, data in state.items():
            # Проверка стандартного отклонения для каждого канала
            std_devs = np.std(data, axis=0)

            # Индексы каналов, которые следует сохранить
            valid_channel_indices = np.where(std_devs > threshold)[0]

            # Сохранение отфильтрованных данных
            if state_name not in filtered_states:
                filtered_states[state_name] = data[:, valid_channel_indices]
            else:
                filtered_states[state_name] = np.concatenate(
                    (filtered_states[state_name], data[:, valid_channel_indices]),
                    axis=0
                )

    filtered_participant_data[participant_id] = filtered_states

## Нормализация данных

In [7]:
from sklearn.preprocessing import StandardScaler

# Инициализация словаря для нормализованных данных
normalized_data = {}

# Нормализация данных
scaler = StandardScaler()

for participant_id, states in participant_data.items():
    normalized_data[participant_id] = {}  # Инициализация словаря для каждого участника

    for state_dict in states:
        for state_name, data in state_dict.items():
            # Применение нормализации
            normalized_data[participant_id][state_name] = scaler.fit_transform(data)

# Теперь normalized_data содержит нормализованные данные для каждого состояния


In [15]:
participant_data

{'201410092013.D091BB44': [{'focused': array([[5390.76923077, 3833.84615385, 4019.48717949, 4656.41025641,
           4125.64102564],
          [5389.23076923, 3830.25641026, 4020.        , 4658.97435897,
           4136.41025641],
          [5385.64102564, 3829.74358974, 4017.94871795, 4655.8974359 ,
           4152.82051282],
          ...,
          [4825.12820513, 3826.15384615, 4016.41025641, 4125.12820513,
           4138.97435897],
          [4814.87179487, 3829.23076923, 4018.46153846, 4119.48717949,
           4142.05128205],
          [4814.35897436, 3828.71794872, 4019.48717949, 4119.48717949,
           4139.48717949]]),
   'unfocused': array([[4815.8974359 , 3828.71794872, 4017.94871795, 4122.56410256,
           4140.        ],
          [4811.28205128, 3829.23076923, 4014.87179487, 4118.97435897,
           4142.56410256],
          [4807.69230769, 3826.15384615, 4014.87179487, 4117.43589744,
           4136.41025641],
          ...,
          [5054.35897436, 3827.179487

## Разметка данных

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Подготовка данных и меток
X = []
y = []

# Объединение данных и меток из всех участников
for participant_id, data in normalized_data.items():
    for state_name, state_data in data.items():
        X.extend(state_data)
        y.extend([state_name] * len(state_data))  # Метки для каждого состояния

# Преобразуем данные в numpy массивы
X = np.array(X)
y = np.array(y)

# Преобразование строковых меток в числовые
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # Пример: ['focused', 'unfocused', 'drowsy'] -> [0, 1, 2]

# Преобразуем метки в one-hot encoding
y_one_hot = to_categorical(y_encoded, num_classes=3)

# Проверяем форму данных
print(f"Форма X: {X.shape}, Форма y_one_hot: {y_one_hot.shape}")

# Разбиваем данные на обучающую и валидационную выборку
X_train, X_val, y_train, y_val = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# Добавляем третий размер (timesteps=1) для LSTM
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))

Форма X: (12512552, 5), Форма y_one_hot: (12512552, 3)


## Разбиение выборки

## Обучение модели на основе LSTM + валидация
*  поскольку она хорошо справляется с временными рядами и может учитывать зависимость состояний во времени.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, BatchNormalization, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Определяем модель LSTM
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(LSTM(32, return_sequences=False))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(3, activation='softmax'))  # 3 класса: focused, unfocused, drowsy

# Компилируем модель
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Определяем callback для ранней остановки
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Обучаем модель
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=64,
    callbacks=[early_stopping]
)

# Оценка модели на валидационной выборке
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=2)
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

Итого результаты LTSM для accuracy: 0.6319 - loss: 0.8743

Это не очень хороший результат, так что возвращаемся в предобработку данных в новом ноутбуке LTSM

## Обучение модели SVM (недостаточно вычислительных мощностей)
(В соревновании рекомендуют именно эту модель, хотя можно было бы попробовать Ensemble или RNN, LSTM, но даже 1 простая SVC отнимает у моего пк все мощности на несколько часов)

```py
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 1. Подготовка данных и меток
X = []
y = []

# Объединение данных и меток из всех участников
for participant_id, data in normalized_data.items():
    for state_name, state_data in data.items():
        X.extend(state_data)
        y.extend([state_name] * len(state_data))  # Метки для каждого состояния

# Преобразуем данные в numpy массивы
X = np.array(X)
y = np.array(y)

# Преобразуем строковые метки в числовые
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # Например, ['focused', 'unfocused', 'drowsy'] -> [0, 1, 2]

# Разбиваем данные на обучающую и валидационную выборку
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
```

```py
# 2. Определяем параметры для GridSearchCV
param_grid = {
    'C': [0.1, 1, 10],        # Коэффициент регуляризации
    'gamma': ['scale', 'auto'],  # Параметр ядра
    'kernel': ['rbf']          # Радиальная базисная функция
}

# 3. Инициализация SVC и поиск по сетке параметров
svc = SVC(random_state=42)
grid_search = GridSearchCV(svc, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# 4. Обучение модели с поиском оптимальных параметров
grid_search.fit(X_train, y_train)

# 5. Вывод наилучших параметров
print(f"Лучшие параметры: {grid_search.best_params_}")

# 6. Оценка модели на валидационной выборке с лучшими параметрами
best_model = grid_search.best_estimator_
y_val_pred = best_model.predict(X_val)

# 7. Оценка модели
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_val, y_val_pred, target_names=label_encoder.classes_))

print("\nConfusion Matrix:")
print(confusion_matrix(y_val, y_val_pred))

# 8. Оценка с кросс-валидацией на обучающей выборке
cv_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='accuracy')
print(f"\nКросс-валидация: Средняя точность = {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
```